In [1]:
import os
import torch 
import torch.nn as nn
from spnc import spnc_anisotropy
import numpy as np
import matplotlib.pyplot as plt
import tqdm as tqdm
import pickle
import spnc_ml as ml


from pathlib import Path

CANDIDATES = [
    
    Path(r"C:\Users\tom\Desktop\Repository"),
    Path(r"C:\Users\Chen\Desktop\Repository"),
]
searchpaths = [p for p in CANDIDATES if p.exists()]

#tuple of repos
repos = ('machine_learning_library',)

from deterministic_mask import fixed_seed_mask, max_sequences_mask
import repo_tools
repo_tools.repos_path_finder(searchpaths, repos)
from single_node_res import single_node_reservoir
import ridge_regression as RR
from linear_layer import *
from mask import binary_mask
from utility import *
from NARMA10 import NARMA10
from datasets.load_TI46_digits import *
import datasets.load_TI46 as TI46
from sklearn.metrics import classification_report


C:\Users\Chen\Desktop\Repository\machine_learning_library\ridge_regression.py:6: SyntaxWarning: invalid escape sequence '\l'
  '''


In [2]:
# 构建储层对象
class ReservoirParams:
    def __init__(self, **kwargs):
            # Reservoir parameters 
            self.h = 0.4
            self.theta_H = 90
            self.k_s_0 = 0
            self.phi = 45
            self.beta_prime = 35.13826524755751

            # Network parameters 
            self.Nvirt = 50
            self.m0 = 0.005288612874870094
            self.bias = True
            self.Nwarmup = 0
            self.verbose_repr = False

            self.params = {
                'theta': 0.34142235979698393,
                'gamma': 0.069274461903986,
                'delay_feedback': 0,
                'Nvirt': self.Nvirt,
                'length_warmup': self.Nwarmup,
                'warmup_sample': self.Nwarmup * self.Nvirt,
                'voltage_noise': False,
                'seed_voltage_noise': 1234,
                'delta_V': 0.1,
                'johnson_noise': False,
                'seed_johnson_noise': 1234,
                'mean_johnson_noise': 0.0000,
                'std_johnson_noise': 0.00001,
                'thermal_noise': False,
                'seed_thermal_noise': 1234,
                'lambda_ou': 1.0,
                'sigma_ou': 0.1
        }

            for key in ['h', 'theta_H', 'k_s_0', 'phi', 'beta_prime', 'Nvirt', 'm0', 'bias', 'Nwarmup']:
                if key in kwargs:
                    setattr(self, key, kwargs[key])

            
            if 'params' in kwargs and isinstance(kwargs['params'], dict):
                self.params.update(kwargs['params'])

    
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            if key in self.params:
                self.params[key] = value
            if not hasattr(self, key) and key not in self.params:
                raise AttributeError(f"ReservoirParams has no attribute or param key '{key}'")
            
    def print_params(self, verbose=False):
        if not verbose:
            print(f"ReservoirParams(h={self.h}, beta_prime={self.beta_prime}, Nvirt={self.Nvirt})")
        else:
            print(f"ReservoirParams detailed info:")
            print(f"  h = {self.h}")
            print(f"  theta_H = {self.theta_H}")
            print(f"  k_s_0 = {self.k_s_0}")
            print(f"  phi = {self.phi}")
            print(f"  beta_prime = {self.beta_prime}")
            print(f"  Nvirt = {self.Nvirt}")
            print(f"  m0 = {self.m0}")
            print(f"  bias = {self.bias}")
            print("  params dictionary:")
            for k, v in self.params.items():
                print(f"    {k}: {v}")

params = ReservoirParams(
        h=0.4, m0=0.006937322149792008, Nvirt=50, beta_prime=27.251620432439488,
        params={'theta': 0.01, 'gamma': 0.3663969812988086, 'Nvirt': 50}
    )
# params = ReservoirParams()

spn_bestCQ = spnc_anisotropy(
        params.h,
        params.theta_H,
        params.k_s_0,
        params.phi,
        params.beta_prime,
        restart=True
    )

# transform = spn.gen_signal_slow_delayed_feedback
transform = spn_bestCQ.gen_signal_slow_delayed_feedback

speakers = ['f1','f2','f3','f4','f5'] 

acc = ml.spnc_TI46(speakers, params.Nvirt, params.m0, params.bias, transform, params.params)
print(acc)

Samples for training:  500
Samples for test:  795
Using MFCC preprocessing
the type of xn is:  <class 'numpy.ndarray'>
the shape of xn is:  (500,)
after normalise, the first feature value is:  [0.04238083 0.41043805 0.64367391 0.50959894 0.74912839 0.58009803
 0.67307435 0.72230249 0.69940684 0.6094933  0.65694511 0.69812443
 0.66056711]
Nin = 13 , Nout =  10 , Nvirt =  50
Deterministic mask will be used
the shape of the mask is:  (50, 13)
after fold the shape of S: (500,)
type of S: <class 'numpy.ndarray'>
J_train.shape, J_train[0].shape   (500,) (20, 50)
S_train.shape, S_train[0].shape (500,) (20, 50)
Linear layer created with shape: (10, 50)
y1h shape: (500,)
y1h[0]: [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0

In [3]:
# 构建储层对象
class ReservoirParams:
    def __init__(self, **kwargs):
            # Reservoir parameters 
            self.h = 0.4
            self.theta_H = 90
            self.k_s_0 = 0
            self.phi = 45
            self.beta_prime = 35.13826524755751

            # Network parameters 
            self.Nvirt = 50
            self.m0 = 0.005288612874870094
            self.bias = True
            self.Nwarmup = 0
            self.verbose_repr = False

            self.params = {
                'theta': 0.34142235979698393,
                'gamma': 0.069274461903986,
                'delay_feedback': 0,
                'Nvirt': self.Nvirt,
                'length_warmup': self.Nwarmup,
                'warmup_sample': self.Nwarmup * self.Nvirt,
                'voltage_noise': False,
                'seed_voltage_noise': 1234,
                'delta_V': 0.1,
                'johnson_noise': False,
                'seed_johnson_noise': 1234,
                'mean_johnson_noise': 0.0000,
                'std_johnson_noise': 0.00001,
                'thermal_noise': False,
                'seed_thermal_noise': 1234,
                'lambda_ou': 1.0,
                'sigma_ou': 0.1
        }

            for key in ['h', 'theta_H', 'k_s_0', 'phi', 'beta_prime', 'Nvirt', 'm0', 'bias', 'Nwarmup']:
                if key in kwargs:
                    setattr(self, key, kwargs[key])

            
            if 'params' in kwargs and isinstance(kwargs['params'], dict):
                self.params.update(kwargs['params'])

    
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            if key in self.params:
                self.params[key] = value
            if not hasattr(self, key) and key not in self.params:
                raise AttributeError(f"ReservoirParams has no attribute or param key '{key}'")
            
    def print_params(self, verbose=False):
        if not verbose:
            print(f"ReservoirParams(h={self.h}, beta_prime={self.beta_prime}, Nvirt={self.Nvirt})")
        else:
            print(f"ReservoirParams detailed info:")
            print(f"  h = {self.h}")
            print(f"  theta_H = {self.theta_H}")
            print(f"  k_s_0 = {self.k_s_0}")
            print(f"  phi = {self.phi}")
            print(f"  beta_prime = {self.beta_prime}")
            print(f"  Nvirt = {self.Nvirt}")
            print(f"  m0 = {self.m0}")
            print(f"  bias = {self.bias}")
            print("  params dictionary:")
            for k, v in self.params.items():
                print(f"    {k}: {v}")

params = ReservoirParams(
        h=0.4, m0=0.006937322149792008, Nvirt=50, beta_prime=27.251620432439488,
        params={'theta': 0.01, 'gamma': 0.3663969812988086, 'Nvirt': 50}
    )
# params = ReservoirParams()

spn_bestCQ_test = spnc_anisotropy(
        params.h,
        params.theta_H,
        params.k_s_0,
        params.phi,
        params.beta_prime,
        restart=True
    )

# transform = spn.gen_signal_slow_delayed_feedback
transform_test = spn_bestCQ_test.gen_signal_slow_delayed_feedback

speakers = ['f1','f2','f3','f4','f5'] 

acc = ml.spnc_TI46_test(speakers, params.Nvirt, params.m0, params.bias, transform_test, params.params)
print(acc)

Samples for training:  500
Samples for test:  795
Using MFCC preprocessing
the type of xn is:  <class 'numpy.ndarray'>
the shape of xn is:  (500,)
after normalise, the first feature value is:  [0.0719358  0.36692493 0.67200084 0.50798323 0.69168498 0.59073393
 0.79947203 0.73438096 0.7459391  0.66082983 0.58053796 0.57003627
 0.6096776 ]
Nin = 13 , Nout =  10 , Nvirt =  50
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.0719358  0.36692493 0.67200084 0.50798323 0.69168498 0.59073393
 0.79947203 0.73438096 0.7459391  0.66082983 0.58053796 0.57003627
 0.6096776 ]
Deterministic mask will be used
after fold the shape of S: (20, 50)
type of S: <class 'numpy.ndarray'>
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.0212491  0.37151116 0.65666329 0.50789505 0.70759341 0.60307721
 0.7324458  0.65934061 0.78191358 0.72379009 0.67452983 0.6

In [ ]:
# 构建储层对象
class ReservoirParams:
    def __init__(self, **kwargs):
            # Reservoir parameters 
            self.h = 0.4
            self.theta_H = 90
            self.k_s_0 = 0
            self.phi = 45
            self.beta_prime = 35.13826524755751

            # Network parameters 
            self.Nvirt = 50
            self.m0 = 0.005288612874870094
            self.bias = True
            self.Nwarmup = 0
            self.verbose_repr = False

            self.params = {
                'theta': 0.34142235979698393,
                'gamma': 0.069274461903986,
                'delay_feedback': 0,
                'Nvirt': self.Nvirt,
                'length_warmup': self.Nwarmup,
                'warmup_sample': self.Nwarmup * self.Nvirt,
                'voltage_noise': False,
                'seed_voltage_noise': 1234,
                'delta_V': 0.1,
                'johnson_noise': False,
                'seed_johnson_noise': 1234,
                'mean_johnson_noise': 0.0000,
                'std_johnson_noise': 0.00001,
                'thermal_noise': False,
                'seed_thermal_noise': 1234,
                'lambda_ou': 1.0,
                'sigma_ou': 0.1
        }

            for key in ['h', 'theta_H', 'k_s_0', 'phi', 'beta_prime', 'Nvirt', 'm0', 'bias', 'Nwarmup']:
                if key in kwargs:
                    setattr(self, key, kwargs[key])

            
            if 'params' in kwargs and isinstance(kwargs['params'], dict):
                self.params.update(kwargs['params'])

    
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            if key in self.params:
                self.params[key] = value
            if not hasattr(self, key) and key not in self.params:
                raise AttributeError(f"ReservoirParams has no attribute or param key '{key}'")
            
    def print_params(self, verbose=False):
        if not verbose:
            print(f"ReservoirParams(h={self.h}, beta_prime={self.beta_prime}, Nvirt={self.Nvirt})")
        else:
            print(f"ReservoirParams detailed info:")
            print(f"  h = {self.h}")
            print(f"  theta_H = {self.theta_H}")
            print(f"  k_s_0 = {self.k_s_0}")
            print(f"  phi = {self.phi}")
            print(f"  beta_prime = {self.beta_prime}")
            print(f"  Nvirt = {self.Nvirt}")
            print(f"  m0 = {self.m0}")
            print(f"  bias = {self.bias}")
            print("  params dictionary:")
            for k, v in self.params.items():
                print(f"    {k}: {v}")

params = ReservoirParams(
        h=0.4, m0=0.008, Nvirt=50, beta_prime=	50.0,
        params={'theta': 0.1564938388583194, 'gamma': 0.04608425844940916, 'Nvirt': 50}
    )
# params = ReservoirParams()

spn_bestMC_test = spnc_anisotropy(
        params.h,
        params.theta_H,
        params.k_s_0,
        params.phi,
        params.beta_prime,
        restart=True
    )

# transform = spn.gen_signal_slow_delayed_feedback
transform_MC_test = spn_bestMC_test.gen_signal_slow_delayed_feedback

speakers = ['f1','f2','f3','f4','f5'] 

acc = ml.spnc_TI46_test(speakers, params.Nvirt, params.m0, params.bias, transform_MC_test, params.params)
print(acc)

Samples for training:  500
Samples for test:  795
Using MFCC preprocessing
the type of xn is:  <class 'numpy.ndarray'>
the shape of xn is:  (500,)
after normalise, the first feature value is:  [0.05815814 0.33279745 0.65150933 0.51965394 0.741396   0.71034558
 0.70175636 0.57727602 0.77066006 0.68254044 0.64568319 0.6630075
 0.63386035]
Nin = 13 , Nout =  10 , Nvirt =  50
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.05815814 0.33279745 0.65150933 0.51965394 0.741396   0.71034558
 0.70175636 0.57727602 0.77066006 0.68254044 0.64568319 0.6630075
 0.63386035]
Deterministic mask will be used
after fold the shape of S: (20, 50)
type of S: <class 'numpy.ndarray'>
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.03025123 0.35084983 0.66543791 0.55732072 0.74763892 0.59689423
 0.68152349 0.6415764  0.75830558 0.69475613 0.73624626 0.694

In [5]:
# 构建储层对象
class ReservoirParams:
    def __init__(self, **kwargs):
            # Reservoir parameters 
            self.h = 0.4
            self.theta_H = 90
            self.k_s_0 = 0
            self.phi = 45
            self.beta_prime = 35.13826524755751

            # Network parameters 
            self.Nvirt = 50
            self.m0 = 0.005288612874870094
            self.bias = True
            self.Nwarmup = 0
            self.verbose_repr = False

            self.params = {
                'theta': 0.34142235979698393,
                'gamma': 0.069274461903986,
                'delay_feedback': 0,
                'Nvirt': self.Nvirt,
                'length_warmup': self.Nwarmup,
                'warmup_sample': self.Nwarmup * self.Nvirt,
                'voltage_noise': False,
                'seed_voltage_noise': 1234,
                'delta_V': 0.1,
                'johnson_noise': False,
                'seed_johnson_noise': 1234,
                'mean_johnson_noise': 0.0000,
                'std_johnson_noise': 0.00001,
                'thermal_noise': False,
                'seed_thermal_noise': 1234,
                'lambda_ou': 1.0,
                'sigma_ou': 0.1
        }

            for key in ['h', 'theta_H', 'k_s_0', 'phi', 'beta_prime', 'Nvirt', 'm0', 'bias', 'Nwarmup']:
                if key in kwargs:
                    setattr(self, key, kwargs[key])

            
            if 'params' in kwargs and isinstance(kwargs['params'], dict):
                self.params.update(kwargs['params'])

    
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            if key in self.params:
                self.params[key] = value
            if not hasattr(self, key) and key not in self.params:
                raise AttributeError(f"ReservoirParams has no attribute or param key '{key}'")
            
    def print_params(self, verbose=False):
        if not verbose:
            print(f"ReservoirParams(h={self.h}, beta_prime={self.beta_prime}, Nvirt={self.Nvirt})")
        else:
            print(f"ReservoirParams detailed info:")
            print(f"  h = {self.h}")
            print(f"  theta_H = {self.theta_H}")
            print(f"  k_s_0 = {self.k_s_0}")
            print(f"  phi = {self.phi}")
            print(f"  beta_prime = {self.beta_prime}")
            print(f"  Nvirt = {self.Nvirt}")
            print(f"  m0 = {self.m0}")
            print(f"  bias = {self.bias}")
            print("  params dictionary:")
            for k, v in self.params.items():
                print(f"    {k}: {v}")

params = ReservoirParams(
        h=0.4, m0=0.008, Nvirt=50, beta_prime=	50.0,
        params={'theta': 0.1564938388583194, 'gamma': 0.04608425844940916, 'Nvirt': 50}
    )
# params = ReservoirParams()

spn_bestMC = spnc_anisotropy(
        params.h,
        params.theta_H,
        params.k_s_0,
        params.phi,
        params.beta_prime,
        restart=True
    )

# transform = spn.gen_signal_slow_delayed_feedback
transform_MC = spn_bestMC.gen_signal_slow_delayed_feedback

speakers = ['f1','f2','f3','f4','f5'] 

acc = ml.spnc_TI46(speakers, params.Nvirt, params.m0, params.bias, transform_MC, params.params)
print(acc)

Samples for training:  500
Samples for test:  795
Using MFCC preprocessing
the type of xn is:  <class 'numpy.ndarray'>
the shape of xn is:  (500,)
after normalise, the first feature value is:  [0.06640167 0.33168489 0.66597501 0.55604356 0.7608783  0.65040307
 0.68961916 0.65941653 0.8105094  0.71205344 0.64121778 0.57799482
 0.5481744 ]
Nin = 13 , Nout =  10 , Nvirt =  50
Deterministic mask will be used
the shape of the mask is:  (50, 13)
after fold the shape of S: (500,)
type of S: <class 'numpy.ndarray'>
J_train.shape, J_train[0].shape   (500,) (20, 50)
S_train.shape, S_train[0].shape (500,) (20, 50)
Linear layer created with shape: (10, 50)
y1h shape: (500,)
y1h[0]: [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0

In [6]:
# 构建储层对象
class ReservoirParams:
    def __init__(self, **kwargs):
            # Reservoir parameters 
            self.h = 0.4
            self.theta_H = 90
            self.k_s_0 = 0
            self.phi = 45
            self.beta_prime = 35.13826524755751

            # Network parameters 
            self.Nvirt = 50
            self.m0 = 0.005288612874870094
            self.bias = True
            self.Nwarmup = 0
            self.verbose_repr = False

            self.params = {
                'theta': 0.34142235979698393,
                'gamma': 0.069274461903986,
                'delay_feedback': 0,
                'Nvirt': self.Nvirt,
                'length_warmup': self.Nwarmup,
                'warmup_sample': self.Nwarmup * self.Nvirt,
                'voltage_noise': False,
                'seed_voltage_noise': 1234,
                'delta_V': 0.1,
                'johnson_noise': False,
                'seed_johnson_noise': 1234,
                'mean_johnson_noise': 0.0000,
                'std_johnson_noise': 0.00001,
                'thermal_noise': False,
                'seed_thermal_noise': 1234,
                'lambda_ou': 1.0,
                'sigma_ou': 0.1
        }

            for key in ['h', 'theta_H', 'k_s_0', 'phi', 'beta_prime', 'Nvirt', 'm0', 'bias', 'Nwarmup']:
                if key in kwargs:
                    setattr(self, key, kwargs[key])

            
            if 'params' in kwargs and isinstance(kwargs['params'], dict):
                self.params.update(kwargs['params'])

    
    def update_params(self, **kwargs):
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            if key in self.params:
                self.params[key] = value
            if not hasattr(self, key) and key not in self.params:
                raise AttributeError(f"ReservoirParams has no attribute or param key '{key}'")
            
    def print_params(self, verbose=False):
        if not verbose:
            print(f"ReservoirParams(h={self.h}, beta_prime={self.beta_prime}, Nvirt={self.Nvirt})")
        else:
            print(f"ReservoirParams detailed info:")
            print(f"  h = {self.h}")
            print(f"  theta_H = {self.theta_H}")
            print(f"  k_s_0 = {self.k_s_0}")
            print(f"  phi = {self.phi}")
            print(f"  beta_prime = {self.beta_prime}")
            print(f"  Nvirt = {self.Nvirt}")
            print(f"  m0 = {self.m0}")
            print(f"  bias = {self.bias}")
            print("  params dictionary:")
            for k, v in self.params.items():
                print(f"    {k}: {v}")

params = ReservoirParams(
        h=0.4, m0=0.008, Nvirt=200, beta_prime=	50.0,
        params={'theta': 0.1564938388583194, 'gamma': 0.04608425844940916, 'Nvirt': 200}
    )
# params = ReservoirParams()

spn_bestMC_200 = spnc_anisotropy(
        params.h,
        params.theta_H,
        params.k_s_0,
        params.phi,
        params.beta_prime,
        restart=True
    )

# transform = spn.gen_signal_slow_delayed_feedback
transform_MC_200 = spn_bestMC_200.gen_signal_slow_delayed_feedback

speakers = ['f1','f2','f3','f4','f5'] 

acc = ml.spnc_TI46_test(speakers, params.Nvirt, params.m0, params.bias, transform_MC_200, params.params)
print(acc)

Samples for training:  500
Samples for test:  795
Using MFCC preprocessing
the type of xn is:  <class 'numpy.ndarray'>
the shape of xn is:  (500,)
after normalise, the first feature value is:  [0.0659038  0.34982107 0.69659939 0.54804241 0.72606452 0.61143246
 0.75491031 0.70784928 0.7514973  0.73599184 0.6156467  0.5543306
 0.50032252]
Nin = 13 , Nout =  10 , Nvirt =  200
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.0659038  0.34982107 0.69659939 0.54804241 0.72606452 0.61143246
 0.75491031 0.70784928 0.7514973  0.73599184 0.6156467  0.5543306
 0.50032252]
Deterministic mask will be used
after fold the shape of S: (20, 200)
type of S: <class 'numpy.ndarray'>
the type of single_xn is:  <class 'numpy.ndarray'>
the shape of single_xn is:  (20, 13)
the first element of single_xn is:  [0.08765252 0.36105636 0.65703994 0.50482878 0.71556183 0.63033698
 0.71748858 0.65675544 0.71533466 0.74446775 0.7815534  0.6